In [62]:
from cProfile import label
import sys
import json
import math
import os
import ast

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt

import os
import seaborn as sns

In [ ]:
data = { 'rank':int, #accending rank of team score (1 lowest)
        'directory':str, #drectory to save files easier'
        'imp_rank':int,#rank of how much they improved ep1vs2 (1 lowest improvemnt or worsening)
        '1': {'rank':int,#same as above for all episodes
                    'meds': dataframe of medics,
                    'engs':dataframe of engs, 
                    'directory':str,#same as above
                    'med_playerid':dataframe of player,
                    'eng_player_id':dataframe of player,
                    'global':team dataframe of episode
                    }



}

In [145]:
directory ='/Users/josecordova/Desktop/Michigan/545/project/TED-Minimap/output/'# '/Users/josecordova/Documents/cmu/TED'
os.chdir(directory)
labels = ['Workload','Skill','Effort']
lims=[1,0.8,1]

data = {}
episodes_score = {}
group_scores = {}
improvement_score = {}
df_eng_global = pd.DataFrame(0, index=np.arange(29), columns=labels)
df_med_global = pd.DataFrame(0, index=np.arange(29), columns=labels)
df_ep1 = pd.DataFrame(0, index=np.arange(29), columns=labels)
df_ep2 = pd.DataFrame(0, index=np.arange(29), columns=labels)
eng_global=0
med_global=0
for group in sorted(os.listdir(directory)):
    if '.' in group: 
        continue
    data[group]={}
    data[group]['1'] = {}
    data[group]['2'] = {}

    curr_directory = directory+group+'/'
    data[group]['directory']=curr_directory

    for episode in sorted(os.listdir(curr_directory)):

        if '.' in episode: 
            continue
        data[group][episode]['directory'] = curr_directory +episode+'/'
        data[group][episode]['label'] = group + '_' +episode
        data[group][episode]['global'] = pd.read_csv(group+'/' +episode +'_global.csv')
        data[group][episode]['global']['time'] = range(0,10*len(data[group][episode]['global']),10)
        data[group][episode]['score']= (50*data[group][episode]['global']['triage_count_red']+data[group][episode]['global']['triage_count_yellow']*30 \
            +data[group][episode]['global']['triage_count_green']*10).sum()
        data[group][episode]['global'].rename(columns = {'process_skill_use_s':'Skill', 'process_effort_s':'Effort','process_workload_burnt':'Workload'}, inplace = True)

        episodes_score[group+'_'+episode] = data[group][episode]['score']

        for proccess in labels:
            data[group][episode][proccess] = data[group][episode]['global'][proccess].sum()

        
        if episode == '1':
            
            df_ep1[labels]+=data[group][episode]['global'][labels]
        else:
            df_ep2[labels]+=data[group][episode]['global'][labels]

        directory_episode = curr_directory+episode+'/'
        data[group][episode]['meds'] = pd.DataFrame(0, index=np.arange(29), columns=labels)
        data[group][episode]['engs'] = pd.DataFrame(0, index=np.arange(29), columns=labels)
        engs = 0
        meds = 0
        for player in os.listdir(directory_episode):
            if not player.endswith(".csv") : 
                continue
            curr_player = directory_episode+player
            data[group][episode][player[:-4]]= pd.read_csv(curr_player)
            data[group][episode][player[:-4]].rename(columns = {'skill_s':'Skill', 'effort':'Effort','workload':'Workload'}, inplace = True)


            if player[0] == 'e':
                engs+=1
                eng_global+=1
                for metric in labels:
                    data[group][episode]['engs'][metric]+=data[group][episode][player[:-4]][metric]
                    df_eng_global[metric]+=data[group][episode][player[:-4]][metric]

            else:
                meds+=1
                med_global+=1
                for metric in labels:
                    data[group][episode]['meds'][metric]+=data[group][episode][player[:-4]][metric]
                    df_med_global[metric]+=data[group][episode][player[:-4]][metric]

                    

        data[group][episode]['engs'][metric]/=engs
        data[group][episode]['meds'][metric]/=meds
    data[group]['score']=data[group]['1']['score'] + data[group]['2']['score']
    group_scores[group]=data[group]['score']

    data[group]['imp_score']=data[group]['2']['score'] - data[group]['1']['score']
    improvement_score[group]=data[group]['imp_score']

df_eng_global /=eng_global
df_med_global/=med_global

df_ep1/=len(data)
df_ep2/=len(data)


episodes_score = dict(sorted(episodes_score.items(), key=lambda item: item[1]))
df = pd.DataFrame(episodes_score.items())
df.to_csv('ep_scores.csv',index=False)
group_scores = dict(sorted(group_scores.items(), key=lambda item: item[1]))
df = pd.DataFrame(group_scores.items())
df.to_csv('group_scores.csv',index=False)
improvement_score = dict(sorted(improvement_score.items(), key=lambda item: item[1]))
df = pd.DataFrame(improvement_score.items())
df.to_csv('imp_scores.csv',index=False)



i=0
for episode in episodes_score:
    data[episode[:-2]][episode[-1]]['rank']=i
    i+=1
i=0
for group in group_scores:
    data[group]['rank']=i
    i+=1
i=0
for group in improvement_score:
    data[group]['impr_rank']=i
    i+=1

In [147]:
metrics = ['Workload','Skill','Effort']
lims=[0.2,0.2,1]

for i, proccess in enumerate(metrics):

    plt.figure()

    plt.plot(df_med_global[proccess],label = 'Medics')
    plt.plot(df_eng_global[proccess],label = 'Engineers')

    plt.xlabel('Time(s)')
    plt.ylabel(proccess)
    plt.title(proccess)
    plt.ylim(top=lims[i],bottom=0)
    plt.legend()

    plt.savefig('Med vs Eng '+proccess,facecolor='w')
    plt.close()


    plt.figure()

    plt.plot(df_ep1[proccess],label = 'Ep. 1')
    plt.plot(df_ep2[proccess],label = 'Ep. 2')

    plt.xlabel('Time(s)')
    plt.ylabel(proccess)
    plt.title(proccess)
    plt.ylim(top=lims[i],bottom=0)
    plt.legend()

    plt.savefig('Ep.1 vs Ep.2 '+proccess+'.png',facecolor='w')
    plt.close()


In [155]:
%%capture

scores=[]
skills=[]
efforts=[]
workloads=[]
labels = []
metrics = ['Workload','Skill','Effort']
plt.ioff()
roles = ['meds','engs']
lims=[0.8,0.8,1]

for group in data.values():
    for episode in group.values():

        if not isinstance(episode, dict):
            continue
        scores.append(episode['score'])
        print(episode['score'])
        skills.append(episode['Skill'])
        efforts.append(episode['Effort'])        
        workloads.append(episode['Workload'])
        labels.append(episode['label'])
        

    for i,proccess in enumerate(metrics):
        plt.figure()

        plt.plot(group['1']['global']['time'],group['1']['global'][proccess],label = 'Episode 1')
        plt.plot(group['2']['global']['time'],group['2']['global'][proccess],label = 'Episode 2')

        plt.xlabel('Time(s)')
        plt.ylabel(proccess)
        plt.title(proccess)
        plt.ylim(top=lims[i],bottom=0)
        plt.legend()

        plt.savefig(group['directory']+proccess,facecolor='w')
        plt.close()
        plt.figure()

        plt.plot(group['1']['global']['time'].head(29),group['1']['meds'][proccess],label = 'Episode 1')
        plt.plot(group['2']['global']['time'].head(29),group['2']['meds'][proccess],label = 'Episode 2')

        plt.xlabel('Time(s)')
        plt.ylabel(proccess)
        plt.title(proccess)
        plt.ylim(top=lims[i],bottom=0)
        plt.legend()

        plt.savefig(group['directory']+'meds_'+proccess,facecolor='w')
        plt.close()


        plt.figure()

        plt.plot(group['2']['global']['time'].head(29),group['2']['engs'][proccess],label = 'Episode 2')

        plt.plot(group['1']['global']['time'].head(29),group['1']['engs'][proccess],label = 'Episode 1')

        plt.xlabel('Time(s)')
        plt.ylabel(proccess)
        plt.title(proccess)
        plt.ylim(top=lims[i],bottom=0)
        plt.legend()

        plt.savefig(group['directory']+'eng_'+proccess,facecolor='w')
        plt.close()


plot_labels = [workloads,skills, efforts]

for i, label in enumerate(plot_labels):
  sns.scatterplot(x=label, y=scores, hue=labels)
  plt.xlabel(metrics[i])
  plt.ylabel('Score')
  plt.title(metrics[i]+' Correlation')
  lgd=plt.legend(loc=(1.04,0), prop={'size': 6},ncol=3)
  
  plt.savefig(metrics[i]+'.png', bbox_extra_artists=(lgd,), bbox_inches='tight',facecolor='w')
  plt.close()

In [118]:
print(len(scores))

0


In [152]:
#### high vs low score episodes
num_epi = 1 
highvslow = {}
highvslow['df_eng_high'] = pd.DataFrame(0, index=np.arange(29), columns=metrics)
highvslow['df_med_low' ]= pd.DataFrame(0, index=np.arange(29), columns=metrics)
highvslow['df_eng_low' ]= pd.DataFrame(0, index=np.arange(29), columns=metrics)
highvslow['df_med_high'] = pd.DataFrame(0, index=np.arange(29), columns=metrics)
eng_high=0
med_low=0

 
for group in data.values():
    for episode in group.values():
        if type(episode)!=dict:
            continue
        if episode['rank'] <= num_epi:
            curr_med = 'df_med_low'
            curr_eng = 'df_eng_low'

        elif episode['rank'] >= (len(data)*2-num_epi):
            curr_med = 'df_med_high'
            curr_eng = 'df_eng_high'            
        else:
            continue

        for player in episode:
            if not '_' in player :
                continue
            
            if player[0]=='e':
                highvslow[curr_eng][metrics]+= episode[player][metrics]

            else:
                highvslow[curr_med]+= episode[player]


for i in highvslow.values():
    i/=(num_epi*2)
for i, proccess in enumerate(metrics):
    plt.figure()

    plt.plot(highvslow['df_med_high'][proccess],label = 'High Med')
    plt.plot(highvslow['df_eng_high'][proccess],label = 'High Eng')

    plt.plot(highvslow['df_med_low'][proccess],label = 'Low Med')
    plt.plot(highvslow['df_eng_low'][proccess],label = 'Low Eng')

    plt.xlabel('Time(s)')
    plt.ylabel(proccess)
    plt.title(proccess)
    plt.ylim(top=lims[i],bottom=0)
    plt.legend()

    plt.savefig('High vs low #='+str(num_epi)+'_'+proccess,facecolor='w')
    plt.close()